In [1]:
import numpy as np

# <u> 3. DQN algorithme avec récupération d'expérience </u> 

On va maintenant implémenter l'algorithme de Deep $Q$-learning, celui-ci suit le principe suivant : 

> <u> <b> Initialisation : </b> </u> On initialise la mémoire $M$ sur les expériences passées sur une liste vide, et on fixe la taille maximale de cette mémoire. 

> <u> <b> A chaque étape tu temps $t\in\mathbb N$, on repète :  </b> </u>
>      
> * On détermine la $Q$-Valeur de l'état courant $S_t$ :
$$ Q^{\pi} : (S_t,A_t) \mapsto \mathbb{E}_{\pi} \displaystyle\left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} | S_t, A_t  \right]$$
>
> * On choisi $A_t$ maximisant $a\mapsto Q(S_t,a)$.
>
> * On calcule la récompense associée $R_t=R(S_t,S_t)$
>
> * On ajoute l'expérience $(S_t, A_t, R_t, S_{t+1})$ à la mémoire $M$ 
>
> * On extrait un batch aléatoire $B\subset M$ de notre mémoire. Puis pour tout $(S_t^B,A_t^B,R_t^B,S_{t+1}^B)$ dans B :
> 
>      - On calcule $Q(S_t^B,A_t^B)$ et $R(S_t^B,A_t^B)+\gamma \max_a Q(S_{t+1}^B,a)$
>
> * On calcule la perte obtenu sur tout le batch : $$ \frac 1 2 \sum_B \displaystyle \left(R(S_t^B,A_t^B)+\gamma \max_a Q(S_{t+1}^B,a) - Q(S_t^B,A_t^B)\right)^2$$
> 
> * On utilise cette perte pour mettre à jour les poids du réseau de neuronnes. 


L'utilisation de mini-batchs permet d'améliorer le processus d'apprentissage en réutilisant des sous-ensembles d'expérience passée pour mettre à jour les poids du réseau de neuronne plutôt que de tout réutiliser. La première raison est le gain de temps de calcul tout en gardant un processus d'apprentissage fiable, permettant en bon compromis entre robustesse et rapidité. 

Maintenant, pour l'implémentation, on procède de la même manière que pour les script précédents : on définit une classe `DQN` qui contiendra notre algorithme d'apprentissage avec expérience. On la définit comme suit : 

<b> a. </b>  On initialise par `__init__` les 3 éléments indispensables de l'algorithme : la mémoire, la capacité de celle-ci et le facteur $\gamma$. On initialise ces deux derniers par défault à $100$ et $0.9$ d'après la littérature. 

<b> b. </b> On construit ensuite une fonction permettant de mettre en mémoire une nouvelle expérience. Celle-ci doit se rappeler de l'expérience $(S_t,A_t,R_t,S_{t+1})$ mais aussi de l'issu de celle-ci, à savoir si `game_over=True` ou `game_over=False`.

<b> c. </b> On souhaite, d'après ce qui a été énoncé précédemment, extraire des batchs d'expériences. On implémente ceci dans la fonction `get_batch`. Celle-ci prends en argument notre modèle de réseau de neuronne défini dans $\texttt{cnn}.\texttt{ipynb}$, et la taille des batchs souhaitée. On initialise ce batch par un tableau numpy de la taille souhaitée (ou toute la mémoire si la taille est plus grande).  

In [7]:
class DQN(object):
    def __init__(self,memory_capacity=100,gamma=0.9):
        self.memory = []
        self.memory_capacity = memory_capacity
        self.discount = gamma

    def save_exp(self,exp,game_over):   #La variable exp=[St,At,Rt,St+1]
        self.memory.append([exp,game_over])
        if len(self.memory) > self.memory_capacity :
            self.memory.pop(0)

    def get_batch(self,model, size_batch):
        # dans la mémoire : M[i]=[[St, At, Rt, St+1], game_over]

        n_memory = len(self.memory)
        n_actions = model.output_shape[-1] #méthode de Seuqential() 

        size_limit = min(n_memory,size_batch)

        states = np.zeros((size_limit,self.memory[0][0][0].shape[1],self.memory[0][0][0].shape[2],self.memory[0][0][0].shape[3]))  # on créer un tableau 4D pour extraire size_limit vecteur 3D car les éléments dans memory sont de la forme [[St,At,Rt,St+1],done] Donc memory[0][0][0]=St et on décide que :
        # St est un vecteur 3D : une dimension pour les lignes de la grilles, une pour les colonnes des grilles
        # St retient l'état de la grille des n dernières actions (d'où la troisième dimension de St 
        targets = np.zeros((size_limit,n_actions))

        batch_set=np.random.randint(0,n_memory,size=size_limit)
        for i,ind in enumerate(batch_set):
            St, At, Rt, Stp1 = self.memory[ind][0]
            game_over = self.memory[ind][1]
            states[i] = St
            targets[i] = model.predict(St)[0]
            Q = np.max(model.predict(Stp1)[0])

            if game_over:
                targets[i,At] = Rt
            else :
                targets[i,At] = Rt + self.discount * Q

        return(states,targets)